In [1]:
from opedia import db

def organismsList(tax, depth1, depth2, cruise_name, cluster_level, size_frac_lower, size_frac_upper, table='tblESV'):
    query = "SELECT [%s], SUM(relative_abundance) AS total FROM %s WHERE " % (tax, table)
    query += "depth BETWEEN %f AND %f AND " % (depth1, depth2)
    query += "cruise_name='%s' AND " % cruise_name
    # removing null, ambiguous, and uncultured taxa labels
    query += "[%s] IS NOT NULL AND " % tax
    query += "[%s] NOT LIKE '%s' AND " % (tax, '%uncultured%')
    query += "[%s] NOT LIKE '%s' AND " % (tax, '%ambiguous%')
    query += "[%s] NOT LIKE '%s' AND " % (tax, '%unidentified%')
    query += "[%s] NOT LIKE '%s' AND " % (tax, '%metagenome%')
    # cluster filter
    query += "cluster_level=%d AND " % cluster_level  
    # size fractions
    query += "size_frac_lower=%f AND " % size_frac_lower  
    if size_frac_upper is not None:
        query += "size_frac_upper=%f " % size_frac_upper 
    else:
        query += "size_frac_upper IS NULL " 
    query += "GROUP BY [%s] ORDER BY total DESC " % tax
    df = db.dbFetch(query)
    df[tax] = df[tax].str.strip()
    return df


In [2]:
tax = ['domain', 'phylum', 'class', 'order', 'family', 'genus', 'species'][5]
depth1 = 20
depth2 = depth1
cruise_name = 'ANT28-5'
iSizeFrac = 0 # Free-living fraction 0.2 - 3 uM
#iSizeFrac = 1 # Small particle-associated 3 - 8 uM 
#iSizeFrac = 2 # Large particle-associated fraction > 8 uM 
size_frac_lower = [0.2, 3, 8][iSizeFrac]
size_frac_upper = [3, 8, None][iSizeFrac]


In [3]:
cluster_level = [89, 92, 96, 97, 98, 99, 100][0] 
df = organismsList(tax, depth1, depth2, cruise_name, cluster_level, size_frac_lower, size_frac_upper)
df.head(10)

,genus,total
0,Prochlorococcus MIT9313,4.297156
1,Candidatus Actinomarina,3.961154
2,Clade Ia,3.116693
3,NS4 marine group,1.607900
4,NS5 marine group,1.073449
5,SUP05 cluster,0.592536
6,Alcanivorax,0.425212
7,Sulfitobacter,0.249726
8,Candidatus Nitrosopelagicus,0.146983
9,Marinoscillum,0.145284


In [4]:
cluster_level = [89, 92, 96, 97, 98, 99, 100][6] 
df = organismsList(tax, depth1, depth2, cruise_name, cluster_level, size_frac_lower, size_frac_upper)
df.head(10)

,genus,total
0,Prochlorococcus MIT9313,4.031382
1,Candidatus Actinomarina,3.961154
2,Clade Ia,1.593928
3,NS5 marine group,1.508555
4,NS4 marine group,1.101254
5,SUP05 cluster,0.313330
6,Synechococcus CC9902,0.260905
7,Marinoscillum,0.145284
8,NS2b marine group,0.141034
9,SAR92 clade,0.128620
